# **Analisi Comparativa di Modelli di Embedding per il Clustering di Testi**

### **Obiettivo del Notebook**

Questo notebook esplora l'efficacia di diversi modelli di embedding per il compito di clustering non supervisionato. L'obiettivo è replicare ed estendere le scoperte dell'articolo scientifico *"Beyond words: a comparative analysis of LLM embeddings for effective clustering"*.

Partendo da un dataset di notizie non etichettate, confronteremo come diversi modelli linguistici trasformano il testo in vettori numerici (embeddings) e come diversi algoritmi di clustering riescono a raggruppare questi vettori in categorie tematiche coerenti.

**Metodologia:**
1.  **Dataset:** Utilizzeremo il dataset **BBC News**, composto da 2225 articoli suddivisi in 5 categorie reali (business, entertainment, politics, sport, tech).
2.  **Modelli di Embedding:** Confronteremo le performance di due modelli molto noti: `all-MiniLM-L6-v2` (leggero e performante), `stsb-bert-base` (un modello BERT specializzato per la similarità semantica) e `BLOOMZ-3B` (modello general-purpose piu' grande degli altri). 
3.  **Algoritmi di Clustering:** Implementeremo un framework flessibile per testare diversi algoritmi. Sono stati scelti, `k-means` (variante ++), `CAEclust`, `Deep k-means`.
4.  **Valutazione:** Misureremo la qualità del clustering utilizzando due metriche standard:
    *   **Accuracy (ACC):** Metrica di precisione relativa al "ground truth".
    *   **Adjusted Rand Index (ARI):** Misura la somiglianza tra i cluster predetti e le etichette reali, correggendo per il caso (valori vicini a 1.0 sono migliori).
    *   **Normalized Mutual Information (NMI):** Valuta la quantità di informazione condivisa tra i cluster predetti e le etichette reali (valori vicini a 1.0 sono migliori).

### **1. Setup dell'Ambiente e Configurazione**
In questa prima cella di codice, importiamo tutte le librerie necessarie e definiamo le variabili di configurazione globali. Centralizzare la configurazione qui rende il notebook più pulito e facile da adattare per futuri esperimenti.

> **Nota:** Assicurati di aver installato tutte le librerie richieste nel tuo ambiente `conda` o `pip`.

In [ ]:
import os
import numpy as np
from itertools import permutations
import warnings
from tqdm.auto import tqdm
import gc


# --- Librerie per Machine Learning e NLP ---
try:
    from sentence_transformers import SentenceTransformer
    from sklearn.cluster import KMeans, SpectralClustering
    from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
    from sklearn.preprocessing import StandardScaler
    import torch
    import torch.nn as nn
    from torch.utils.data import DataLoader, TensorDataset


except ImportError as e:
    print(f"ERRORE: Libreria mancante -> {e.name}")
    print("Assicurati di aver installato 'scikit-learn' e 'sentence-transformers'.")

# Ignora avvisi non critici per una migliore leggibilità dell'output
warnings.filterwarnings('ignore', category=FutureWarning)

# --- CONFIGURAZIONE PRINCIPALE ---

# Percorso del dataset
DATASET_PATH = "./dataset"

# Modelli di embedding
MODELS_TO_TEST = {
    'MiniLM': 'all-MiniLM-L6-v2',
    'BERT_STS': 'stsb-bert-base', 
    'BLOOMZ_3B': 'bigscience/bloomz-3b' # Identificatore ufficiale su Hugging Face
}

### **2. Funzioni di Supporto**
Per mantenere il codice modulare e riutilizzabile, definiamo due funzioni principali:
1.  `load_data_from_folders`: Legge i file `.txt` dalle sottocartelle del dataset e li carica in memoria insieme alle loro etichette reali (il nome della cartella).
2.  `get_or_create_embeddings`: Una funzione cruciale che gestisce la generazione degli embedding. Implementa un meccanismo di **caching**: se gli embedding per un modello sono già stati calcolati e salvati su un file `.npy`, li carica direttamente; altrimenti, li genera da zero e li salva per le esecuzioni future. Questo fa risparmiare moltissimo tempo.

In [4]:

def load_data_from_folders(path):
    """Carica i testi e le etichette, applicando il lowercase."""
    texts, labels = [], []
    if not os.path.exists(path): return None, None
    categories = sorted([d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))])
    if not categories: return None, None
    print(f"Trovate {len(categories)} categorie: {categories}")
    for category in categories:
        category_path = os.path.join(path, category)
        for filename in sorted(os.listdir(category_path)):
            if filename.endswith('.txt'):
                with open(os.path.join(path, category, filename), 'r', encoding='latin-1') as f:
                    texts.append(f.read().lower())
                    labels.append(category)
    print(f"Caricati e pre-processati (lowercase) {len(texts)} documenti.")
    return texts, labels



def get_embeddings_with_chunking(model, texts, chunk_size=512, overlap=64):
    """
    Genera embedding per documenti lunghi dividendoli in chunk e facendo la media.
    Converte esplicitamente il tipo di dati prima di passare a NumPy.
    """
    final_embeddings = []
    for doc in tqdm(texts, desc="Processing documents with chunking"):
        if not doc.strip():
            final_embeddings.append(np.zeros(model.get_sentence_embedding_dimension()))
            continue
        tokens = model.tokenizer.encode(doc, add_special_tokens=False)
        chunk_embeddings = []
        for i in range(0, len(tokens), chunk_size - overlap):
            chunk_tokens = tokens[i:i + chunk_size]
            if not chunk_tokens: continue
            chunk_text = model.tokenizer.decode(chunk_tokens)
            if not chunk_text.strip(): continue
            # Calcoliamo l'embedding del chunk (sarà in bfloat16)
            chunk_embedding = model.encode(chunk_text, convert_to_tensor=True, show_progress_bar=False)
            chunk_embeddings.append(chunk_embedding)
        
        if chunk_embeddings:
            # Calcoliamo la media (il risultato è ancora in bfloat16)
            mean_embedding = torch.mean(torch.stack(chunk_embeddings), dim=0)
            
            # Convertiamo il tensore in float32 PRIMA di passarlo a cpu() e numpy()
            final_embeddings.append(mean_embedding.float().cpu().numpy())
        else:
            final_embeddings.append(np.zeros(model.get_sentence_embedding_dimension()))

    return np.array(final_embeddings)




def get_or_create_embeddings(texts, model_name, friendly_name):
    """Genera gli embedding con una logica di device pulita e robusta, usando bfloat16 per i modelli grandi."""
    embedding_file = f"bbc_embeddings_{friendly_name}_lowercase.npy"
    if os.path.exists(embedding_file):
        print(f"Caricamento degli embedding per '{friendly_name}' dal file locale: {embedding_file}")
        return np.load(embedding_file)
    else:
        print(f"Generazione degli embedding per '{friendly_name}' con il modello '{model_name}'...")
        
        model_kwargs = {}
        st_kwargs = {}
        needs_chunking = False
        device = "cuda" if torch.cuda.is_available() else "cpu"

        if 'bloomz' in model_name.lower():
             print("Applicazione ottimizzazioni per modello BLOOMZ (bfloat16 e device_map)...")
             model_kwargs['torch_dtype'] = torch.bfloat16 
             model_kwargs['device_map'] = 'auto'
             needs_chunking = True
        else:
             st_kwargs['device'] = device

        if 'device_map' in model_kwargs:
            st_kwargs = {}

        model = SentenceTransformer(model_name, model_kwargs=model_kwargs, **st_kwargs)
        
        if needs_chunking:
            embeddings = get_embeddings_with_chunking(model, texts)
        else:
            batch_size = 32
            print(f"Inizio encoding con batch_size = {batch_size}...")
            embeddings = model.encode(texts, show_progress_bar=True, batch_size=batch_size)
        
        # Aggiungiamo un controllo per i valori NaN/inf prima di salvare
        if np.any(np.isnan(embeddings)) or np.any(np.isinf(embeddings)):
            print("ATTENZIONE: Trovati valori NaN o Inf negli embedding. Rimuovendoli...")
            # Sostituisce NaN e Inf con zero
            embeddings = np.nan_to_num(embeddings, nan=0.0, posinf=0.0, neginf=0.0)

        del model
        gc.collect()
        torch.cuda.empty_cache()
        print("Pulizia della memoria GPU completata.")
        
        print(f"Salvataggio degli embedding su file: {embedding_file}")
        np.save(embedding_file, embeddings)
        return np.array(embeddings)
    
def calculate_accuracy(true_labels, predicted_labels):
    """
    Calcola l'accuratezza del clustering (ACC) trovando la migliore mappatura
    tra le etichette predette e quelle reali usando le permutazioni.
    """
    
    # --- CORREZIONE: Converti gli input in array NumPy per confronti robusti ---
    true_labels = np.asarray(true_labels)
    predicted_labels = np.asarray(predicted_labels)
    
    # 1. Trova le etichette uniche
    true_unique_labels = np.unique(true_labels)
    pred_unique_labels = np.unique(predicted_labels)
    
    # Gestisce il caso in cui il clustering trovi un numero diverso di cluster
    if len(true_unique_labels) != len(pred_unique_labels):
        print(f"ATTENZIONE: Il numero di cluster reali ({len(true_unique_labels)}) è diverso dal numero di cluster predetti ({len(pred_unique_labels)}). L'accuratezza potrebbe non essere significativa.")
        # In questo caso, la mappatura uno-a-uno non è possibile, restituiamo 0.0
        return 0.0

    n_clusters = len(pred_unique_labels)
    
    # 2. Crea la matrice di contingenza
    contingency = np.zeros((n_clusters, n_clusters), dtype=np.int64)
    for true_idx, true_label in enumerate(true_unique_labels):
        for pred_idx, pred_label in enumerate(pred_unique_labels):
            contingency[true_idx, pred_idx] = np.sum((true_labels == true_label) & (predicted_labels == pred_label))

    # 3. Itera su tutte le possibili mappature (permutazioni)
    best_accuracy = 0.0
    
    for perm in permutations(range(n_clusters)):
        # `perm` mappa l'indice della riga (etichetta reale) all'indice della colonna (etichetta predetta)
        current_correct_count = contingency[range(n_clusters), perm].sum()
        
        accuracy = current_correct_count / np.sum(contingency)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            
    return best_accuracy

### **3. Definizione degli Algoritmi di Clustering**
Questa cella rende il nostro framework di test estremamente flessibile. La funzione `get_clustering_algorithms` restituisce un dizionario di algoritmi di `scikit-learn` pronti per essere utilizzati.

**Per testare un nuovo algoritmo di clustering, è sufficiente:**
1.  Importarlo nella Cella 1 (Snippet 3).
2.  Aggiungere una nuova voce al dizionario `algorithms` in questa cella.

Iniziamo con `K-Means++`, dove "++" si riferisce a un metodo di inizializzazione intelligente che migliora la stabilità e la qualità dei cluster finali.

In [6]:

# --- 1. Definizione dell'Autoencoder Profondo  ---
class Autoencoder(nn.Module):
    """
    Un autoencoder più profondo con regularizzazione Dropout.
    """
    def __init__(self, input_dim, bottleneck_dim=64, dropout_rate=0.2):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 256), nn.ReLU(), nn.Dropout(dropout_rate),
            nn.Linear(256, 128), nn.ReLU(), nn.Dropout(dropout_rate),
            nn.Linear(128, bottleneck_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(bottleneck_dim, 128), nn.ReLU(), nn.Dropout(dropout_rate),
            nn.Linear(128, 256), nn.ReLU(), nn.Dropout(dropout_rate),
            nn.Linear(256, input_dim)
        )
    def forward(self, x):
        return self.decoder(self.encoder(x))

# --- 2. Wrapper per l'algoritmo CAEclust ---
class CAEclustWrapper:
    def __init__(self, n_clusters, n_autoencoders=5, bottleneck_dim=128, epochs=100, dropout_rate=0.2):
        self.n_clusters = n_clusters
        self.n_autoencoders = n_autoencoders
        self.bottleneck_dim = bottleneck_dim
        self.epochs = epochs
        self.dropout_rate = dropout_rate
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"CAEclust (PyTorch) inizializzato: {self.n_autoencoders} DAEs, bottleneck a {self.bottleneck_dim}, dropout {self.dropout_rate}, {self.epochs} epoche. Device: {self.device}")

    def fit_predict(self, embeddings):
        scaler = StandardScaler()
        scaled_embeddings = scaler.fit_transform(embeddings)
        input_dim = scaled_embeddings.shape[1]
        embeddings_tensor = torch.FloatTensor(scaled_embeddings).to(self.device)
        trained_encoders = []

        print(f"\nInizio addestramento di {self.n_autoencoders} Denoising Autoencoder...")
        for i in range(self.n_autoencoders):
            print(f"  Training DAE {i+1}/{self.n_autoencoders}...")
            model = Autoencoder(input_dim, self.bottleneck_dim, self.dropout_rate).to(self.device)
            criterion = nn.MSELoss()
            optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
            noise_factor = 0.05
            noisy_embeddings_tensor = embeddings_tensor + noise_factor * torch.randn_like(embeddings_tensor)
            dataset = TensorDataset(noisy_embeddings_tensor, embeddings_tensor)
            data_loader = DataLoader(dataset, batch_size=64, shuffle=True)
            model.train()
            for epoch in range(self.epochs):
                for noisy_inputs, clean_targets in data_loader:
                    outputs = model(noisy_inputs)
                    loss = criterion(outputs, clean_targets)
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
            trained_encoders.append(model.encoder)

        consensus_features_tensor = torch.FloatTensor(scaled_embeddings).to(self.device)
        consensus_representations = []
        with torch.no_grad():
            for encoder in trained_encoders:
                encoder.eval()
                encoded_data = encoder(consensus_features_tensor).cpu().numpy()
                consensus_representations.append(encoded_data)
        
        consensus_features = np.concatenate(consensus_representations, axis=1)
        consensus_features = StandardScaler().fit_transform(consensus_features)
        print(f"Nuova forma delle feature di consenso: {consensus_features.shape}")
        
        print("Applicazione di Spectral Clustering...")
        clusterer = SpectralClustering(n_clusters=self.n_clusters, random_state=42, affinity='nearest_neighbors', n_jobs=-1)
        labels = clusterer.fit_predict(consensus_features)
        return labels

# --- 3. Wrapper per l'algoritmo Deep K-Means ---
class DeepKMeansWrapper:
    def __init__(self, n_clusters, bottleneck_dim=64, epochs=100, gamma=0.1):
        self.n_clusters = n_clusters
        self.bottleneck_dim = bottleneck_dim
        self.epochs = epochs
        self.gamma = gamma
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Deep K-Means (PyTorch) inizializzato: bottleneck a {self.bottleneck_dim}, {self.epochs} epoche, gamma={self.gamma}. Device: {self.device}")

    def _calculate_clustering_loss(self, latent_vectors, centroids):
        distances = torch.sum((latent_vectors.unsqueeze(1) - centroids) ** 2, dim=2)
        min_distances, _ = torch.min(distances, dim=1)
        return torch.mean(min_distances)

    def fit_predict(self, embeddings):
        scaler = StandardScaler()
        scaled_embeddings = scaler.fit_transform(embeddings)
        input_dim = scaled_embeddings.shape[1]
        embeddings_tensor = torch.FloatTensor(scaled_embeddings).to(self.device)
        data_loader = DataLoader(TensorDataset(embeddings_tensor), batch_size=64, shuffle=True)

        model = Autoencoder(input_dim, self.bottleneck_dim).to(self.device)
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
        reconstruction_criterion = nn.MSELoss()

        # Pre-training
        print("\nInizio pre-training dell'Autoencoder (20 epoche)...")
        for epoch in range(20):
            for batch in data_loader:
                inputs, = batch
                outputs = model(inputs)
                loss = reconstruction_criterion(outputs, inputs)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
        
        print("Inizializzazione dei centroidi con K-Means...")
        with torch.no_grad():
            model.eval()
            initial_latent_space = model.encoder(embeddings_tensor).cpu().numpy()
        kmeans = KMeans(n_clusters=self.n_clusters, init='k-means++', n_init=10, random_state=42)
        kmeans.fit(initial_latent_space)
        centroids = torch.FloatTensor(kmeans.cluster_centers_).to(self.device)

        # Training Congiunto (DKM)
        print("Inizio addestramento congiunto (Deep K-Means)...")
        model.train()
        for epoch in range(self.epochs):
            for batch in data_loader:
                inputs, = batch
                latent_vectors = model.encoder(inputs)
                reconstructed_vectors = model.decoder(latent_vectors)
                recon_loss = reconstruction_criterion(reconstructed_vectors, inputs)
                clust_loss = self._calculate_clustering_loss(latent_vectors, centroids)
                total_loss = recon_loss + self.gamma * clust_loss
                optimizer.zero_grad()
                total_loss.backward()
                optimizer.step()
        
        # Assegnazione finale
        print("Assegnazione finale ai cluster...")
        with torch.no_grad():
            model.eval()
            final_latent_space = model.encoder(embeddings_tensor).cpu().numpy()
        
        final_kmeans = KMeans(n_clusters=self.n_clusters, init=centroids.cpu().numpy(), n_init=1, random_state=42)
        labels = final_kmeans.fit_predict(final_latent_space)
        return labels

# --- 4. Funzione per ottenere tutti gli algoritmi ---
def get_clustering_algorithms(n_clusters):
    """
    Restituisce un dizionario di algoritmi di clustering, inclusi i metodi deep.
    """
    algorithms = {
        'K-Means++': KMeans(n_clusters=n_clusters, init='k-means++', random_state=42, n_init=10),
        'CAEclust (Deep)': CAEclustWrapper(n_clusters=n_clusters),
        'Deep K-Means': DeepKMeansWrapper(n_clusters=n_clusters)
    }
    return algorithms

### **4. Esperimento 1 - Modello `all-MiniLM-L6-v2`**

Iniziamo il nostro primo esperimento con **MiniLM**. Questo modello è noto per il suo eccellente equilibrio tra velocità, dimensioni ridotte e alta performance, rendendolo una baseline molto forte.

Il codice nella cella successiva eseguirà i seguenti passaggi:
1.  Carica i dati testuali (se non già presenti in memoria).
2.  Ottiene gli embedding per il modello MiniLM.
3.  Esegue un ciclo su tutti gli algoritmi definiti nella Cella 3.
4.  Calcola e stampa le metriche di performance ARI e NMI per ogni algoritmo.

In [7]:

# 1. Caricamento dei dati (da eseguire solo una volta se non già fatto)
if 'texts' not in locals():
    texts, true_labels = load_data_from_folders(DATASET_PATH)
    n_clusters = len(set(true_labels))

# 2. Test per MiniLM
print("\n" + "="*70)
print("===== INIZIO TEST MODELLO: MiniLM =====")
minilm_embeddings = get_or_create_embeddings(texts, MODELS_TO_TEST['MiniLM'], 'MiniLM')
print(f"Forma della matrice di embedding: {minilm_embeddings.shape}")

# 3. Applica tutti gli algoritmi di clustering definiti
clustering_algorithms_to_run = get_clustering_algorithms(n_clusters)
for algo_name, algorithm in clustering_algorithms_to_run.items():
    print(f"\n--- Applicazione dell'algoritmo: {algo_name} ---")
    predicted_labels = algorithm.fit_predict(minilm_embeddings)
    
    ari = adjusted_rand_score(true_labels, predicted_labels)
    nmi = normalized_mutual_info_score(true_labels, predicted_labels)
    acc = calculate_accuracy(true_labels, predicted_labels)
    
    print(f"Risultati per {algo_name}: Accuracy (ACC): {acc:.4f}, ARI = {ari:.4f}, NMI = {nmi:.4f}")
print("="*70)

Trovate 5 categorie: ['.git', 'business', 'entertainment', 'politics', 'sport']
Caricati e pre-processati (lowercase) 1824 documenti.

===== INIZIO TEST MODELLO: MiniLM =====
Caricamento degli embedding per 'MiniLM' dal file locale: bbc_embeddings_MiniLM_lowercase.npy
Forma della matrice di embedding: (1824, 384)
CAEclust (PyTorch) inizializzato: 5 DAEs, bottleneck a 128, dropout 0.2, 100 epoche. Device: cuda
Deep K-Means (PyTorch) inizializzato: bottleneck a 64, 100 epoche, gamma=0.1. Device: cuda

--- Applicazione dell'algoritmo: K-Means++ ---
Risultati per K-Means++: Accuracy (ACC): 0.9731, ARI = 0.9308, NMI = 0.9000

--- Applicazione dell'algoritmo: CAEclust (Deep) ---

Inizio addestramento di 5 Denoising Autoencoder...
  Training DAE 1/5...
  Training DAE 2/5...
  Training DAE 3/5...
  Training DAE 4/5...
  Training DAE 5/5...
Nuova forma delle feature di consenso: (1824, 640)
Applicazione di Spectral Clustering...
Risultati per CAEclust (Deep): Accuracy (ACC): 0.6634, ARI = 0.588

### **5. Esperimento 2 - Modello `stsb-bert-base`**
Ora ripetiamo l'esperimento utilizzando un modello **BERT** che è stato specificamente addestrato (fine-tuned) per compiti di **Semantic Textual Similarity (STS)**. A differenza di un BERT generico, questo modello è ottimizzato per produrre embedding in cui la distanza tra i vettori riflette fedelmente la similarità semantica dei testi.

Questo confronto è cruciale per capire quanto la specializzazione di un modello influenzi la sua efficacia in un compito di clustering tematico.

In [ ]:
# --- Esperimento 2: BERT (STS) ---

print("\n" + "="*70)
print("===== INIZIO TEST MODELLO: BERT (STS) =====")

# 1. Carica i dati freschi per questo esperimento
texts, true_labels = load_data_from_folders(DATASET_PATH)
if texts is None:
    print("Caricamento dati fallito. Salto dell'esperimento.")
else:
    n_clusters = len(set(true_labels))

    # 2. Ottieni gli embedding per BERT (STS)
    bert_embeddings = get_or_create_embeddings(texts, MODELS_TO_TEST['BERT_STS'], 'BERT_STS')
    print(f"Forma della matrice di embedding: {bert_embeddings.shape}")

    # 3. Applica tutti gli algoritmi di clustering definiti
    clustering_algorithms_to_run = get_clustering_algorithms(n_clusters)
    for algo_name, algorithm in clustering_algorithms_to_run.items():
        print(f"\n--- Applicazione dell'algoritmo: {algo_name} ---")
        predicted_labels = algorithm.fit_predict(bert_embeddings)
        
        # Verifica di coerenza prima di calcolare i punteggi
        if len(true_labels) == len(predicted_labels):
            ari = adjusted_rand_score(true_labels, predicted_labels)
            nmi = normalized_mutual_info_score(true_labels, predicted_labels)
            acc = calculate_accuracy(true_labels, predicted_labels)
            print(f"Risultati per {algo_name}: Accuracy (ACC): {acc:.4f}, ARI = {ari:.4f}, NMI = {nmi:.4f}")
        else:
            print(f"ERRORE: Incoerenza nelle dimensioni! Etichette reali: {len(true_labels)}, Etichette predette: {len(predicted_labels)}")
print("="*70)

### **6. Esperimento 3 - Modello `BLOOMZ-3B`**


Dopo aver analizzato i modelli specializzati, è il momento di testare un modello di una classe completamente diversa: **`bigscience/bloomz-3b`**. Con i suoi 3 miliardi di parametri, questo modello rappresenta un approccio "generalista" e su larga scala.

Poiché BLOOMZ non è stato primariamente progettato per produrre embedding di documenti, utilizzeremo la strategia standard del **mean pooling** per estrarre un singolo vettore da tutti i suoi output.

**Nota Tecnica:** L'esecuzione di questo modello ha richiesto ottimizzazioni significative a causa delle sue dimensioni. Abbiamo dovuto implementare il caricamento in mezza precisione (`bfloat16`), la distribuzione automatica del modello tra CPU e GPU (`device_map="auto"`) e una strategia di **chunking** per gestire i documenti lunghi senza esaurire la memoria della GPU.


In [ ]:

print("\n" + "="*70)
print("===== INIZIO TEST MODELLO: BLOOMZ-3B =====")

# 1. Carica i dati freschi
texts, true_labels = load_data_from_folders(DATASET_PATH)
n_clusters = len(set(true_labels))

# 2. Ottieni gli embedding per BLOOMZ-3B
bloomz_embeddings = get_or_create_embeddings(texts, MODELS_TO_TEST['BLOOMZ_3B'], 'BLOOMZ_3B')
print(f"Forma della matrice di embedding: {bloomz_embeddings.shape}")

# 3. Applica tutti gli algoritmi di clustering definiti
clustering_algorithms_to_run = get_clustering_algorithms(n_clusters)
for algo_name, algorithm in clustering_algorithms_to_run.items():
    print(f"\n--- Applicazione dell'algoritmo: {algo_name} ---")
    predicted_labels = algorithm.fit_predict(bloomz_embeddings)
    
    ari = adjusted_rand_score(true_labels, predicted_labels)
    nmi = normalized_mutual_info_score(true_labels, predicted_labels)
    acc = calculate_accuracy(true_labels, predicted_labels)
    
    print(f"Risultati per {algo_name}: Accuracy (ACC): {acc:.4f}, ARI = {ari:.4f}, NMI = {nmi:.4f}")
print("="*70)